### Module to download stock price using Yahoo Finance

Run this only once so we don't need worry about hit the daily download limits when running/debug code

In [66]:
from datetime import datetime, timedelta
import yfinance as yf   #https://pypi.org/project/yfinance/

In [67]:
# turn off debug to get symbol and period for download
debug = False

In [68]:
%run merge-data.ipynb

<ipython-input-68-11ba52ca5917>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['company']=df.apply(lambda x: getCompanyName(x.sponsor_collaborators, x.lead_sponsor, x.collaborators), axis=1)
<ipython-input-68-11ba52ca5917>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['symbol']=df.company.apply(lambda x: findSymbol(x, ticker_all))


In [10]:
#debug = True

# question related source data overview
if (debug):
    print('source data shape: ', df.shape)
    print('total unique symbols:', df.symbol.nunique())
    print('symbol list', df.symbol.unique())

In [11]:
# start_date is when the study is open for participant recruitment
# first_posted is when study record was first released at ClinicalTrials.gov
sdf=df[['symbol','start_date','first_posted','last_update_posted']]
sdf['start_date']=pd.to_datetime(sdf['start_date'])
sdf['start_date']=np.where(pd.isna(sdf.start_date), sdf['first_posted'], sdf['start_date'])

# last_update_posted is the most recent date when updates to the study record were released at ClinicalTrials.gov
sdf['last_update_posted']=pd.to_datetime(sdf['last_update_posted'])

<ipython-input-11-60b7346303ee>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf['start_date']=pd.to_datetime(sdf['start_date'])
<ipython-input-11-60b7346303ee>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf['start_date']=np.where(pd.isna(sdf.start_date), sdf['first_posted'], sdf['start_date'])
<ipython-input-11-60b7346303ee>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [13]:
# get unique symbol and max time period to download price data
symbols = sdf[['symbol','start_date','last_update_posted']].groupby('symbol').agg(
    mindate=('start_date','min'),
    maxdate=('last_update_posted','max')
)
symbols = symbols.reset_index()

In [14]:
# functions to download symbol price during a period
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
def yfdownload(symbol, startdate, enddate):
    try:
        df = yf.download(symbol,start=startdate, end=enddate, progress=False, auto_adjust=True)
    except: 
        df = pd.DataFrame()
    return df

def getSymbolPriceData(symbol, mindate, maxdate):    
    startdate = mindate - timedelta(weeks=2) # two weeks before
    enddate = maxdate + timedelta(weeks=2)  # two weeks after  
    
    df = pd.DataFrame()
    # some company went public after clinical trial, so no price for some dates
    for single_date in daterange(startdate, enddate):
        #print(single_date.strftime("%Y-%m-%d"))
        df = yfdownload(symbol,single_date, enddate)        
        if (df.shape[0]>0):
            # add symbol back to the data frame
            df['symbol']=symbol
            break
    
    return df

In [15]:
# getSymbolPriceData testing:
if (debug):
    print(getSymbolPriceData('ABC','2020-02-10','2020-02-10'))

In [16]:
# download all symbol price data and save for late use since we may hit the daily free download limits
allprice = pd.DataFrame()
for row in symbols.itertuples(name='ticker'):
    allprice = allprice.append(getSymbolPriceData(row.symbol, row.mindate, row.maxdate))

allprice.to_csv('allprice' + str(datetime.now())[-7:-1] + '.csv')


1 Failed download:
- ANGN: Data doesn't exist for startDate = 1594958400, endDate = 1597723200

1 Failed download:
- ANGN: Data doesn't exist for startDate = 1595044800, endDate = 1597723200

1 Failed download:
- ANGN: Data doesn't exist for startDate = 1595131200, endDate = 1597723200

1 Failed download:
- ANGN: Data doesn't exist for startDate = 1595217600, endDate = 1597723200

1 Failed download:
- ANGN: Data doesn't exist for startDate = 1595304000, endDate = 1597723200

1 Failed download:
- ANGN: Data doesn't exist for startDate = 1595390400, endDate = 1597723200

1 Failed download:
- ANGN: Data doesn't exist for startDate = 1595476800, endDate = 1597723200

1 Failed download:
- ANGN: Data doesn't exist for startDate = 1595563200, endDate = 1597723200

1 Failed download:
- ANGN: Data doesn't exist for startDate = 1595649600, endDate = 1597723200

1 Failed download:
- ANGN: Data doesn't exist for startDate = 1595736000, endDate = 1597723200

1 Failed download:
- ANGN: Data doesn't

In [17]:
## errors from yf:
# - ANGN: Data doesn't exist for startDate = 1598760000, endDate = 1598932800 (2020-07-29  2020-08-31)
# - CERT: Data doesn't exist for startDate = 1604552400, endDate = 1607576400 (2020-11-06  2020-12-09)
# - GLBL: Data doesn't exist for startDate = 1612328400, endDate = 1615352400 (2021-02-04  2021-03-09)
# - LBPS: Data doesn't exist for startDate = 1592971200, endDate = 1595995200 (2020-06-25  2020-07-28)
# - LGVN: Data doesn't exist for startDate = 1604898000, endDate = 1607922000 (2020-11-10  2020-12-13)
# - VALN: Data doesn't exist for startDate = 1614661200, endDate = 1617681600 (2021-03-03  2021-04-05)


In [29]:
# find out more about symbols has no price
if (debug):
    print('all symbol price data shape:', allprice.shape)
    tickers = set(df.symbol.unique()) - set(allprice.symbol.unique())
    print('no price symbols: ', tickers)
    
    tickerlist = ''
    for ticker in tickers:
        tickerlist = tickerlist + ' ' + ticker
        #hist = ticker.history(period="max")
        #print(hist)
    
    #tickers = yf.Tickers("ANGN CERT GLBL LBPS LGVN VALN" ) #['BPTS', 'CVAC']
    tickers = yf.Tickers(tickerlist.strip()) 
    
    # hist=tickers.tickers['ANGN'].history(period="max").head(1)
    # hist=tickers.tickers['BPTS'].history(period="max").head(1)
    # hist=tickers.tickers['CVAC'].history(period="max").head(1)
    # hist=tickers.tickers['LBPS'].history(period="max").head(1)
    # print(hist)

In [65]:
# allprice0=pd.read_csv('allprice.csv', index_col=0, parse_dates=True)
# allprice1=pd.read_csv('allprice.13337.csv', index_col=0, parse_dates=True)
# print(allprice0.shape, allprice1.shape)

# a = pd.concat([allprice0, allprice1]).drop_duplicates()
# a.to_csv('allprice.csv')